In [ ]:
import pandas as pd
import os

pd.set_option("display.max_columns", None)

In [ ]:
# %%
base_path = input("Enter the file base path: ")

final_path = input("Enter the final file path: ")

In [ ]:
# %%
files = os.listdir(base_path)

In [ ]:
combined = pd.DataFrame()

for file in files:
    df = pd.read_parquet(f"{base_path}/{file}")

    combined = pd.concat([combined, df], ignore_index=True, axis=0)
    # print(file)

    # print(f'max ecoregions length is {df["observed_ecoregions"].str.len().max()}')
    # print(f'max rights length is {df["rights"].str.len().max()}')

In [ ]:
combined

In [ ]:
# REMOVE NONE VALUES FROM RIGHTS COLUMN
# test["rights"] = test["rights"].apply(lambda x: [i for i in x if i is not None])

In [ ]:
# ecoregions

combined_grouped = combined.groupby(
    ["scientific_name", "kingdom", "phylum", "class", "order", "family", "genus"],
    as_index=False,
    dropna=False,
).apply(
    lambda x: pd.Series(
        {
            "observed_ecoregions": list(
                [item for sublist in x["observed_ecoregions"] for item in sublist]
            ),
            "rights": list([item for sublist in x["rights"] for item in sublist]),
        }
    )
)

In [ ]:
combined_grouped

In [ ]:
print(
    f'max ecoregions length is {combined_grouped["observed_ecoregions"].str.len().max()}'
)
print(f'max rights length is {combined_grouped["rights"].str.len().max()}')

In [ ]:
# ecoregions
# SLOW, TOOK ABOUT AN HOUR FOR 300,000+ ROWS
for i in range(len(combined_grouped)):
    print(f"index: {i}")
    # print(len(test["observed_ecoregions"].iloc[i]))
    cat = []
    rights_cat = []
    for j in range(len(combined_grouped["observed_ecoregions"].iloc[i])):
        if combined_grouped["observed_ecoregions"].iloc[i][j] not in cat:
            cat.append(combined_grouped["observed_ecoregions"].iloc[i][j])
            rights_cat.append(combined_grouped["rights"].iloc[i][j])
        # print(test["observed_ecoregions"].iloc[i][j])
        # for k in test["observed_ecoregions"].iloc[i][j]:
        # print(k)
        # if k not in cat:
        #     cat.append(k)
    # if len(cat) != len(test["observed_ecoregions"].iloc[i]):
    # print(f'index: {i}')

    # print(f'observed: {test["observed_ecoregions"].iloc[i]}')
    # print(f'cat: {cat}')
    if len(combined_grouped["rights"].iloc[i]) != len(
        combined_grouped["observed_ecoregions"].iloc[i]
    ):
        print(f"index: {i}")
        print("rights")

    combined_grouped["observed_ecoregions"].iloc[i] = cat
    combined_grouped["rights"].iloc[i] = rights_cat

In [ ]:
# freshwater

combined_grouped = combined.groupby(
    ["scientific_name", "kingdom", "phylum", "class", "order", "family", "genus"],
    as_index=False,
    dropna=False,
).apply(
    lambda x: pd.Series(
        {
            "freshwater_ecoregions": list(
                set(
                    [item for sublist in x["freshwater_ecoregions"] for item in sublist]
                )
            ),
        }
    )
)

# soil

# combined_grouped = combined.groupby(
#     ["scientific_name", "kingdom", "phylum", "class", "order", "family", "genus"],
#     as_index=False, dropna=False,
# ).apply(
#     lambda x: pd.Series(
#         {
#             "soil_id": list([item for sublist in x["soil_id"] for item in sublist])
#             "specific_soil_name": list(
#                 [
#                     item
#                     for sublist in x["specific_soil_name"]
#                     for item in sublist
#                     if item is not None
#                 ]
#             ),
#             "dominant_soil_name": list(
#                 [
#                     item
#                     for sublist in x["dominant_soil_name"]
#                     for item in sublist
#                     if item is not None
#                 ]
#             ),
#         }
#     )
# )
# combined_grouped = combined.groupby(
#     ["scientific_name", "kingdom", "phylum", "class", "order", "family", "genus"],
#     as_index=False, dropna=False,
# ).agg(
#     {
#         "soil_id": lambda x: list([item for sublist in x for item in sublist]),
#     }
# )

# import dask.dataframe as dd

# # create a Dask DataFrame
# dask_df = dd.from_pandas(combined, npartitions=10)


# def get_unique_values(group):
#     return pd.Series(
#         {
#             "soil_id": list([item for sublist in group["soil_id"] for item in sublist]),
#             "specific_soil_name": list(
#                 [item for sublist in group["specific_soil_name"] for item in sublist]
#             ),
#             "dominant_soil_name": list(
#                 [item for sublist in group["dominant_soil_name"] for item in sublist]
#             ),
#         }
#     )


# combined_grouped = (
#     dask_df.groupby(
#         ["scientific_name", "kingdom", "phylum", "class", "order", "family", "genus"]
#     )
#     .apply(get_unique_values)
#     .reset_index()
#     .compute()
# )

In [ ]:
duplicates = combined_grouped[
    combined_grouped.duplicated(["scientific_name"], keep=False)
]

duplicates

In [ ]:
combined_grouped.to_parquet(f"{final_path}/combined.parquet")